In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms

In [ ]:
transform = transforms.Compose(
    [transforms.RandomCrop(32, padding=4),
     transforms.RandomHorizontalFlip(),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 32

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

100%|██████████| 170M/170M [00:02<00:00, 62.0MB/s]


In [ ]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self, input_channels=3, input_size=32):
        super().__init__()

        # Convolutional layers
        self.features = nn.Sequential(
            nn.Conv2d(input_channels, 32, kernel_size=3, stride=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),

            nn.Conv2d(32, 64, kernel_size=3, stride=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),

            nn.Conv2d(64, 128, kernel_size=3, stride=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),

            nn.Conv2d(128, 256, kernel_size=3, stride=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
        )

        # Pass a dummy input through the convolutional layers
        self._dummy_input = torch.zeros(1, input_channels, input_size, input_size)  # batch size = 1
        self._dummy_output = self.features(self._dummy_input)
        fc_input_size = self._dummy_output.view(1, -1).size(1)

        self.classifier = nn.Sequential(
            nn.Linear(fc_input_size, 120),
            nn.ReLU(),
            nn.Dropout(p=0.2),
            nn.Linear(120, 84),
            nn.ReLU(),
            nn.Linear(84, 10)
        )


    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

net = Net()

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
net.to(device)

Net(
  (features): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
    (5): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
    (9): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU()
    (11): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1))
    (12): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (13): ReLU()
  )
  (classifier): Sequential(
    (0): Linear(in_features=1024, out_features=120, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.2, in

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01, weight_decay=5e-4,  momentum=0.9)

In [ ]:
epochs = 10

for epoch in range(epochs):
    # Training mode
    net.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for i, data in enumerate(trainloader, 0):
        inputs, labels = data

        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        running_loss += loss.item()
        if i % 200 == 199:
            print(f'[Epoch {epoch + 1}, Batch {i + 1:5d}] '
                  f'Loss: {running_loss / 200:.3f} '
                  f'Train Acc: {100 * correct / total:.2f}%')
            running_loss = 0.0
            correct = 0
            total = 0

    # Validation after each epoch
    net.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for val_data in testloader:
            val_inputs, val_labels = val_data
            val_outputs = net(val_inputs)
            val_loss += criterion(val_outputs, val_labels).item()

            _, val_predicted = torch.max(val_outputs.data, 1)
            val_total += val_labels.size(0)
            val_correct += (val_predicted == val_labels).sum().item()

    val_loss /= len(testloader)
    val_acc = 100 * val_correct / val_total
    print(f'[Epoch {epoch + 1}] Validation Loss: {val_loss:.3f} '
          f'Validation Acc: {val_acc:.2f}%')

    # Save model checkpoint
    PATH = './cifar_net.pth'
    torch.save(net.state_dict(), PATH)


[Epoch 1, Batch   200] Loss: 0.755 Train Acc: 74.27%
[Epoch 1, Batch   400] Loss: 0.740 Train Acc: 75.02%
[Epoch 1, Batch   600] Loss: 0.740 Train Acc: 74.80%
[Epoch 1, Batch   800] Loss: 0.720 Train Acc: 75.30%
[Epoch 1, Batch  1000] Loss: 0.726 Train Acc: 74.95%
[Epoch 1, Batch  1200] Loss: 0.734 Train Acc: 74.77%
[Epoch 1, Batch  1400] Loss: 0.726 Train Acc: 74.66%
[Epoch 1] Validation Loss: 0.694 Validation Acc: 76.33%
[Epoch 2, Batch   200] Loss: 0.698 Train Acc: 75.89%
[Epoch 2, Batch   400] Loss: 0.712 Train Acc: 75.28%
[Epoch 2, Batch   600] Loss: 0.716 Train Acc: 75.81%
[Epoch 2, Batch   800] Loss: 0.714 Train Acc: 75.30%
[Epoch 2, Batch  1000] Loss: 0.724 Train Acc: 75.53%
[Epoch 2, Batch  1200] Loss: 0.740 Train Acc: 74.73%
[Epoch 2, Batch  1400] Loss: 0.733 Train Acc: 74.89%
[Epoch 2] Validation Loss: 0.741 Validation Acc: 74.90%
[Epoch 3, Batch   200] Loss: 0.693 Train Acc: 76.25%
[Epoch 3, Batch   400] Loss: 0.709 Train Acc: 74.97%
[Epoch 3, Batch   600] Loss: 0.713 Train

In [ ]:
net = Net()
net.load_state_dict(torch.load(PATH, weights_only=True))

correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in testloader:
        images, labels = data
        # calculate outputs by running images through the network
        outputs = net(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct / total} %')

Accuracy of the network on the 10000 test images: 73.84 %
